In [1]:
import numpy as np
import yaml
import os
import json
import torch

from IPython.display import clear_output
from PIL import Image

In [2]:
!pip install ultralytics
clear_output()

from ultralytics import YOLO

In [3]:
class Config:
    def __init__(self, params: dict):

        for key, value in params.items():
            setattr(self, key, value)
        
        print('Config ready')

# **Constants for notebook**

In [4]:
os.environ['WANDB_DISABLED'] = 'true'

In [5]:
# Train params
MODEL_NAME = './yolov8x.pt'
IMAGE_SIZE = 640
BATCH_SIZE = 16
NUM_EPOCHS = 100
YAML_PATH = ''
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Generate YAML file params
RELATIVE_TRAIN_PATH = '/kaggle/input/yolosvarka-augmented/YOLOSvarka_augmented/images/train' # Relative to 'PATH_TO_DATASET'
RELATIVE_TEST_PATH =  ''# Relative to 'PATH_TO_DATASET'
REALATIVE_VAL_PATH = '/kaggle/input/yolosvarka-augmented/YOLOSvarka_augmented/images/val' # Relative to 'PATH_TO_DATASET'
DICT_WITH_CLASS_NAMES = ["adj", "int", "geo", "pro", "non"]

In [6]:
params = {'model_name': MODEL_NAME,
          'image_size': IMAGE_SIZE,
          'batch_size': BATCH_SIZE,
          'num_epochs': NUM_EPOCHS,
          'yaml_path': YAML_PATH,
          'device': DEVICE}

config = Config(params=params)

Config ready


In [7]:
model = YOLO(config.model_name)

100%|██████████| 131M/131M [00:00<00:00, 295MB/s]


# **Data**

If needed we can create yaml file

Dataset structure for train YOLO:

```
dataset/
├── train/
│   ├── images/
│   └── labels/
└── val/
    ├── images/
    └── labels/
```

labels - txt file for each image with content ```class box_center_x_n box_center_y_n wn hn```

In [8]:
import yaml

In [9]:
content = {
    #'path': PATH_TO_DATASET, 
    'train': RELATIVE_TRAIN_PATH,
    #'test': RELATIVE_TEST_PATH,
    'val': REALATIVE_VAL_PATH,
    'nc': len(DICT_WITH_CLASS_NAMES),
    'names': DICT_WITH_CLASS_NAMES
}

with open('./data.yaml', 'w') as file:
    yaml.dump(content, file, default_flow_style=False)

print("data.yaml file created successfully.")


data.yaml file created successfully.


# **Train**

In [10]:
os.environ['WANDB_SILENT']="true"

In [11]:
model.train(data='/kaggle/working/data.yaml',
            imgsz=config.image_size,
            epochs=config.num_epochs,
            batch=config.batch_size,
            device=config.device,
            name='Svarka',
            degrees=5,
            hsv_h=0.03,
            hsv_s=1,
            hsv_v=0.8,
            mosaic=1,
            flipud=0.5,
            erasing=0.4,
            translate=0,
            scale=0, 
            mixup=0.1,
            copy_paste=0.6,
            )

Ultralytics YOLOv8.2.32 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=./yolov8x.pt, data=/kaggle/working/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=Svarka, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

100%|██████████| 755k/755k [00:00<00:00, 8.44MB/s]
2024-06-15 03:59:47,686	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-15 03:59:49,251	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 74.0MB/s]


AMP: checks passed ✅


...

                   all        434       1462      0.912      0.894      0.928      0.552



100 epochs completed in 4.050 hours.
Optimizer stripped from runs/detect/Svarka/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/Svarka/weights/best.pt, 136.7MB

Validating runs/detect/Svarka/weights/best.pt...
Ultralytics YOLOv8.2.32 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 268 layers, 68128383 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:12<00:00,  1.15it/s]


                   all        434       1462      0.921      0.892      0.929      0.553
                   adj        145        520      0.879      0.844       0.86      0.438
                   int        192        250      0.919      0.865      0.928      0.526
                   geo        281        380       0.94      0.904      0.945      0.621
                   pro        118        189      0.957      0.939      0.968       0.56
                   non         91        123      0.909      0.911      0.946      0.617
Speed: 0.2ms preprocess, 16.5ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/Svarka


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ded7b3654b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [12]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [13]:
zip_dir('/kaggle/working/')

/kaggle/working/directory.zip